In [1]:
from tqdm.notebook import tqdm

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_colwidth', None)
from sklearn.metrics import f1_score,make_scorer

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

import catboost as cb
from sklearn.decomposition import PCA,TruncatedSVD

In [2]:
df=pd.read_csv(r'D:\Python_Projects\hakaton\dataset\emotional_dataset_full.csv')
df.head()

,Unnamed: 0,text,target,prepared_text,prepared_text_lemm,prepared_text_lemm_nostop
0,0,"Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D",1,да все таки он немного похож на него но мой мальчик все равно лучше :D,да все таки он немного похожий на него но мой мальчик все равный хороший :d,таки немного похожий мальчик равный хороший :d
1,1,RT @KatiaCheh: Ну ты идиотка) я испугалась за тебя!!!,1,ну ты идиотка я испугалась за тебя ),ну ты идиотка я испугаться за тебя ),идиотка испугаться )
2,2,"RT @digger2912: ""Кто то в углу сидит и погибает от голода, а мы ещё 2 порции взяли, хотя уже и так жрать не хотим"" :DD http://t.co/GqG6iuE2…",1,кто то в углу сидит и погибает от голода а мы ещё порции взяли хотя уже и так жрать не хотим :D,кто то в угол сидеть и погибать от голод а мы ещё порция взять хотя уже и так жрать не хотеть :d,угол сидеть погибать голод ещё порция взять хотя жрать хотеть :d
3,3,"@irina_dyshkant Вот что значит страшилка :D\nНо блин,посмотрев все части,у тебя создастся ощущение,что авторы курили что-то :D",1,вот что значит страшилка но блин посмотрев все части у тебя создастся ощущение что авторы курили что то :D :D,вот что значить страшилка но блин посмотреть весь часть у тебя создаться ощущение что автор курить что то :d :d,значить страшилка блин посмотреть весь часть создаться ощущение автор курить :d :d
4,4,ну любишь или нет? — Я не знаю кто ты бля:D http://t.co/brf9eNg1U6,1,ну любишь или нет я не знаю кто ты бля :D,ну любить или нет я не знать кто ты бля :d,любить знать бля :d


In [3]:
df.dropna(inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 420271 entries, 0 to 421119
Data columns (total 6 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   Unnamed: 0                 420271 non-null  int64 
 1   text                       420271 non-null  object
 2   target                     420271 non-null  int64 
 3   prepared_text              420271 non-null  object
 4   prepared_text_lemm         420271 non-null  object
 5   prepared_text_lemm_nostop  420271 non-null  object
dtypes: int64(2), object(4)
memory usage: 22.4+ MB


### Подготовка данных к модели. Создание тренировочной и тестовой выборок.

In [6]:
def reduce_rows(df):
    reduced_df = df.groupby('target').apply(lambda x: x.sample(n=50000)).reset_index(drop=True)
    return reduced_df

reduced_df = reduce_rows(df)

C:\Users\Dasha\AppData\Local\Temp\ipykernel_13784\2447628208.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reduced_df = df.groupby('target').apply(lambda x: x.sample(n=50000)).reset_index(drop=True)


In [7]:
reduced_df['target'].value_counts()

target
-1    50000
 0    50000
 1    50000
Name: count, dtype: int64

In [8]:
X = reduced_df['prepared_text_lemm_nostop']
y = reduced_df['target']

In [9]:
RANDOM_STATE = 42
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2,stratify=y, random_state=RANDOM_STATE)

### Векторизация и создание корпуса с помощью Tf-IDF и SVD

In [10]:
tf_idf = TfidfVectorizer()
X_train = tf_idf.fit_transform(X_train)
X_test = tf_idf.transform(X_test)

In [11]:
pca = TruncatedSVD(n_components=1000) 
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [15]:
import pickle 

In [16]:
pkl_filename = "emotion_TFIDF.pkl" 
with open(pkl_filename, 'wb') as file: 
    pickle.dump(tf_idf, file)

In [17]:
pkl_filename = "emotion_SVD.pkl" 
with open(pkl_filename, 'wb') as file: 
    pickle.dump(pca, file)

### Обучение моделей

**Логистическая регрессия**

In [14]:
logreg = LogisticRegression(solver='liblinear', max_iter=500,penalty='l1',
                              random_state=RANDOM_STATE,C=1)
logreg.fit(X_train, y_train)
y_pred=logreg.predict(X_test)
print('Значение метрики для Логистической Регрессии:', f1_score(y_pred,y_test, average='macro'))

pkl_filename = "logreg_emotion_TFIDF.pkl" 
with open(pkl_filename, 'wb') as file: 
    pickle.dump(logreg, file)

' logreg = LogisticRegression(solver=\'liblinear\', max_iter=500,penalty=\'l1\',\n                              random_state=RANDOM_STATE,C=1)\nlogreg.fit(X_train, y_train)\ny_pred=logreg.predict(X_test)\nprint(\'Значение метрики для Логистической Регрессии:\', f1_score(y_pred,y_test, average=\'macro\'))\n\npkl_filename = "logreg_emotion_TFIDF.pkl" \nwith open(pkl_filename, \'wb\') as file: \n    pickle.dump(logreg, file)  '

**Метод ближайших соседей - KNN**

In [15]:
knn = KNeighborsClassifier(algorithm='auto',metric='cosine',weights='uniform',n_neighbors=5,n_jobs=-1)
knn.fit(X_train, y_train)
y_pred=knn.predict(X_test)
print('Значение метрики для Метода ближайших соседей:', f1_score(y_pred,y_test, average='macro'))

pkl_filename = "knn_emotion_TFIDF.pkl" 
with open(pkl_filename, 'wb') as file: 
    pickle.dump(knn, file)

' knn = KNeighborsClassifier(algorithm=\'auto\',metric=\'cosine\',weights=\'uniform\',n_neighbors=5,n_jobs=-1)\nknn.fit(X_train, y_train)\ny_pred=knn.predict(X_test)\nprint(\'Значение метрики для Метода ближайших соседей:\', f1_score(y_pred,y_test, average=\'macro\'))\n\npkl_filename = "knn_emotion_TFIDF.pkl" \nwith open(pkl_filename, \'wb\') as file: \n    pickle.dump(knn, file)  '

**Наивный Байес**

In [17]:
gauss = GaussianNB()
gauss.fit(X_train_pca, y_train)

y_pred=gauss.predict(X_test_pca)
print('Значение метрики для Наивного Байеса:', f1_score(y_pred,y_test, average='macro')) 

pkl_filename = "gauss_emotion_TFIDF.pkl" 
with open(pkl_filename, 'wb') as file: 
    pickle.dump(gauss, file)

' gauss = GaussianNB()\ngauss.fit(X_train_pca, y_train)\n\ny_pred=gauss.predict(X_test_pca)\nprint(\'Значение метрики для Наивного гаусса:\', f1_score(y_pred,y_test, average=\'macro\')) \n\npkl_filename = "gauss_emotion_TFIDF.pkl" \nwith open(pkl_filename, \'wb\') as file: \n    pickle.dump(gauss, file) '

**Линейный Дискриминантный Анализ**

In [18]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_pca, y_train)
y_pred=lda.predict(X_test_pca)
print('Значение метрики для LDA:', f1_score(y_pred,y_test, average='macro'))  


pkl_filename = "lda_emotion_TFIDF.pkl" 
with open(pkl_filename, 'wb') as file: 
    pickle.dump(lda, file)

' lda = LinearDiscriminantAnalysis()\nlda.fit(X_train_pca, y_train)\ny_pred=lda.predict(X_test_pca)\nprint(\'Значение метрики для LDA:\', f1_score(y_pred,y_test, average=\'macro\'))  \n\n\npkl_filename = "lda_emotion_TFIDF.pkl" \nwith open(pkl_filename, \'wb\') as file: \n    pickle.dump(lda, file) '

**Catboost**

In [19]:
catb = cb.CatBoostClassifier(iterations=100, learning_rate=0.1, depth=10)
catb.fit(X_train, y_train)

y_pred=catb.predict(X_test)
print('Значение метрики для CatBoost:', f1_score(y_pred,y_test, average='macro'))

pkl_filename = "catb_emotion_TFIDF.pkl" 
with open(pkl_filename, 'wb') as file: 
    pickle.dump(catb, file)

' catb = cb.CatBoostClassifier(iterations=100, learning_rate=0.1, depth=10)\ncatb.fit(X_train, y_train)\n\ny_pred=catb.predict(X_test)\nprint(\'Значение метрики для CatBoost:\', f1_score(y_pred,y_test, average=\'macro\'))\n\npkl_filename = "catb_emotion_TFIDF.pkl" \nwith open(pkl_filename, \'wb\') as file: \n    pickle.dump(catb, file) '

**MLPClassifier**

In [20]:
mlp = MLPClassifier(hidden_layer_sizes=(5, 5, 5,5,5), max_iter=100, solver='sgd', activation='relu')
mlp.fit(X_train, y_train)
y_pred=mlp.predict(X_test)
print('Значение метрики для MLP:', f1_score(y_pred,y_test, average='macro'))

pkl_filename = "mlp_emotion_TFIDF.pkl" 
with open(pkl_filename, 'wb') as file: 
    pickle.dump(mlp, file)

' mlp = MLPClassifier(hidden_layer_sizes=(5, 5, 5,5,5), max_iter=100, solver=\'sgd\', activation=\'relu\')\nmlp.fit(X_train, y_train)\ny_pred=mlp.predict(X_test)\nprint(\'Значение метрики для MLP:\', f1_score(y_pred,y_test, average=\'macro\'))\n\npkl_filename = "mlp_emotion_TFIDF.pkl" \nwith open(pkl_filename, \'wb\') as file: \n    pickle.dump(mlp, file) '